In [1]:
import pandas as pd
import mysql.connector
import json

In [6]:
def query_to_pd(query, columns):
    cursor.execute(query)
    data = cursor.fetchall()
    df = pd.DataFrame(data, columns=columns)
    return df.reset_index(drop=True)

In [7]:
with open('database_init.json') as file:
    db_config = json.load(file)
db_name = "NBA_DB"
try:
    temp_config = db_config.copy()
    cnxn = mysql.connector.connect(**temp_config)
    cnxn.database = db_name
    cursor = cnxn.cursor()
    query = """SELECT PLAYERS_DETAIL.FULL_NAME,
                      PLAYERS_DETAIL.HEIGHT
               FROM PLAYERS_DETAIL
                        JOIN AWARDS ON AWARDS.player_id = PLAYERS_DETAIL.player_id
               WHERE AWARDS.SEASON IN (2019, 2020, 2021, 2022, 2023, 2024);"""
    query_to_pd(query, ['full_name', 'height']).to_csv('csv1_1.csv', index=False)
    query = """
            SELECT FULL_NAME,
                   HEIGHT,
                   SEASON
            FROM PLAYERS_DETAIL
                     JOIN TOP_PLAYERS ON TOP_PLAYERS.player_id = PLAYERS_DETAIL.player_id
            WHERE TOP_PLAYERS.season IN (2019, 2020, 2021, 2022, 2023, 2024)
            GROUP BY FULL_NAME, HEIGHT, SEASON;"""
    query_to_pd(query, ['full_name', 'height', 'season']).to_csv('csv1_2.csv', index=False)
    query = """
            select FULL_NAME, HEIGHT, (YEAR - YEAR_MIN) AS EXPERIENCE, YEAR AS SEASON
            from WINNER_TEAMS
                join PLAYERS_DETAIL
            on WINNER_TEAMS.player_id = PLAYERS_DETAIL.player_id
            where WINNER_TEAMS.YEAR in (2023, 2024)
            group by FULL_NAME, HEIGHT, (YEAR - YEAR_MIN), YEAR; \
            """
    query_to_pd(query, ['full_name', 'height', 'experience', 'season']).to_csv('csv2_1.csv', index=False)
    query = """
            SELECT FULL_NAME, HEIGHT, (SEASON - YEAR_MIN) AS EXPERIENCE, SEASON
            FROM TOP_PLAYERS
                     JOIN PLAYERS_DETAIL ON TOP_PLAYERS.player_id = PLAYERS_DETAIL.player_id
            WHERE TOP_PLAYERS.SEASON IN (2023, 2024)
              AND TOP_PLAYERS.Rk <= 15
            GROUP BY FULL_NAME, HEIGHT, (SEASON - YEAR_MIN), SEASON; \
            """
    query_to_pd(query, ['full_name', 'height', 'experience', 'season']).to_csv('csv2_2.csv', index=False)
except Exception as e:
    print(e)
finally:
    if 'cnxn' in locals() and cnxn.is_connected():
        cursor.close()
        cnxn.close()
        print("Connection Closed.")

Connection Closed.
